# 816-JHU_COVID19_US

**[Work in progress]**

This notebook is a template for downloading an analysis-ready dataset of COVID-19 confirmed cases and deaths by the US county level from Johns Hopkins University.

Data are presented in two formats
1. Original JHU format with each day as a separate column
2. Time series format with cummulative confirmed cases and death by day

Data source: [COVID-19 Data Repository by the Center for Systems Science and Engineering (CSSE) at Johns Hopkins University](https://github.com/CSSEGISandData/COVID-19)

Author: Peter Rose (pwrose@ucsd.edu)

In [1]:
import os
from pathlib import Path
import pandas as pd
import dateutil

In [2]:
pd.options.display.max_rows = None  # display all rows
pd.options.display.max_columns = None  # display all columsns

In [3]:
def split_by_day(df, day, label):
    day_df = df[['stateFips', 'countyFips', 'admin1', 'admin2', 'lat', 'long', day]].copy()
    day_df.rename(columns={day: label}, inplace=True)
    day_df['date'] = day
    return day_df

### Process US cummulative confirmed cases

In [4]:
confirmed = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv", dtype='str')

In [5]:
confirmed = confirmed.fillna('')

Use consistent column names

In [6]:
confirmed.rename(columns={'Province_State': 'admin1', 'Admin2': 'admin2', 
                          'Lat': 'lat', 'Long_': 'long'}, inplace=True)

Trim off .0 from FIPS code if present

In [7]:
confirmed['FIPS'] = confirmed['FIPS'].str.replace('\.0', '')

In [8]:
confirmed.head()

,UID,iso2,iso3,code3,FIPS,admin2,admin1,Country_Region,lat,long,Combined_Key,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20,5/20/20,5/21/20,5/22/20,5/23/20,5/24/20,5/25/20,5/26/20,5/27/20,5/28/20,5/29/20,5/30/20,5/31/20,6/1/20,6/2/20,6/3/20,6/4/20,6/5/20,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20,6/12/20,6/13/20,6/14/20,6/15/20,6/16/20,6/17/20,6/18/20,6/19/20,6/20/20,6/21/20,6/22/20,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20,7/3/20,7/4/20,7/5/20,7/6/20,7/7/20,7/8/20,7/9/20,7/10/20,7/11/20,7/12/20,7/13/20,7/14/20,7/15/20,7/16/20,7/17/20,7/18/20,7/19/20,7/20/20,7/21/20,7/22/20,7/23/20,7/24/20,7/25/20,7/26/20,7/27/20,7/28/20,7/29/20,7/30/20,7/31/20,8/1/20,8/2/20,8/3/20,8/4/20,8/5/20,8/6/20,8/7/20,8/8/20,8/9/20,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20,8/19/20,8/20/20,8/21/20,8/22/20,8/23/20,8/24/20,8/25/20,8/26/20,8/27/20,8/28/20,8/29/20,8/30/20,8/31/20,9/1/20,9/2/20,9/3/20,9/4/20,9/5/20,9/6/20,9/7/20,9/8/20,9/9/20,9/10/20,9/11/20,9/12/20,9/13/20,9/14/20,9/15/20,9/16/20,9/17/20,9/18/20,9/19/20,9/20/20,9/21/20,9/22/20,9/23/20,9/24/20,9/25/20,9/26/20,9/27/20,9/28/20,9/29/20,9/30/20,10/1/20,10/2/20,10/3/20,10/4/20,10/5/20,10/6/20,10/7/20,10/8/20,10/9/20,10/10/20,10/11/20,10/12/20,10/13/20
0,84001001,US,USA,840,1001,Autauga,Alabama,US,32.53952745,-86.64408227,"Autauga, Alabama, US",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,4,6,6,6,6,6,7,8,10,12,12,12,12,12,12,15,17,19,19,19,23,24,26,26,25,26,28,30,32,33,36,36,37,39,40,43,44,42,45,48,53,53,58,61,67,68,74,84,91,93,104,103,110,110,120,127,136,143,149,155,159,168,189,192,205,212,216,221,233,239,239,241,248,259,265,272,282,295,312,323,331,357,364,367,369,394,405,425,428,436,447,463,473,482,492,497,521,530,545,553,560,583,607,610,636,643,653,662,676,698,715,733,743,765,773,810,823,838,845,863,877,891,900,909,932,941,949,963,972,988,1010,1024,1030,1030,1050,1065,1086,1169,1174,1162,1188,1196,1196,1206,1218,1219,1235,1241,1240,1255,1264,1266,1286,1286,1281,1284,1296,1309,1345,1348,1354,1345,1349,1355,1371,1377,1383,1385,1398,1413,1420,1432,1442,1447,1463,1619,1624,1664,1673,1690,1691,1714,1715,1738,1757,1764,1773,1785,1787,1791,1798,1805,1818,1828,1831,1839,1852,1863,1882,1898,1905,1911,1924
1,84001003,US,USA,840,1003,Baldwin,Alabama,US,30.72774991,-87.72207058,"Baldwin, Alabama, US",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,2,2,2,3,4,4,5,5,10,15,18,19,20,24,28,29,29,38,42,44,56,59,66,71,72,87,91,101,103,109,112,117,123,132,143,147,147,161,168,171,174,174,175,181,187,188,189,196,205,208,216,222,224,227,231,243,244,254,254,260,262,270,269,271,273,274,274,277,282,281,282,283,290,292,292,292,293,296,304,313,320,325,331,343,353,361,364,368,374,377,386,398,405,415,422,435,449,462,500,539,559,626,663,686,735,828,846,864,894,980,1039,1114,1170,1207,1277,1341,1396,1499,1579,1669,1796,1914,1990,2077,2164,2423,2473,2620,2665,2727,2793,2865,2984,3056,3095,3175,3210,3263,3322,3380,3443,3502,3544,3651,3670,3710,3744,3791,3839,3877,3

### Keep only cases with fips codes

In [9]:
county_confirmed = confirmed.query("admin2 != ''").query("FIPS != ''").copy()

In [10]:
# state fips code: 2 character with '0'-padding, e.g. 06
county_confirmed['stateFips'] = county_confirmed['FIPS'].str[:-3]
county_confirmed['stateFips'] = county_confirmed['stateFips'].apply(lambda s: '0' + s if len(s) == 1 else s)
county_confirmed['countyFips'] = county_confirmed['FIPS'].str[-3:]
county_confirmed['lat'] = county_confirmed['lat'].astype('float')
county_confirmed['long'] = county_confirmed['long'].astype('float')

In [11]:
county_confirmed.head()

,UID,iso2,iso3,code3,FIPS,admin2,admin1,Country_Region,lat,long,Combined_Key,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20,5/20/20,5/21/20,5/22/20,5/23/20,5/24/20,5/25/20,5/26/20,5/27/20,5/28/20,5/29/20,5/30/20,5/31/20,6/1/20,6/2/20,6/3/20,6/4/20,6/5/20,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20,6/12/20,6/13/20,6/14/20,6/15/20,6/16/20,6/17/20,6/18/20,6/19/20,6/20/20,6/21/20,6/22/20,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20,7/3/20,7/4/20,7/5/20,7/6/20,7/7/20,7/8/20,7/9/20,7/10/20,7/11/20,7/12/20,7/13/20,7/14/20,7/15/20,7/16/20,7/17/20,7/18/20,7/19/20,7/20/20,7/21/20,7/22/20,7/23/20,7/24/20,7/25/20,7/26/20,7/27/20,7/28/20,7/29/20,7/30/20,7/31/20,8/1/20,8/2/20,8/3/20,8/4/20,8/5/20,8/6/20,8/7/20,8/8/20,8/9/20,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20,8/19/20,8/20/20,8/21/20,8/22/20,8/23/20,8/24/20,8/25/20,8/26/20,8/27/20,8/28/20,8/29/20,8/30/20,8/31/20,9/1/20,9/2/20,9/3/20,9/4/20,9/5/20,9/6/20,9/7/20,9/8/20,9/9/20,9/10/20,9/11/20,9/12/20,9/13/20,9/14/20,9/15/20,9/16/20,9/17/20,9/18/20,9/19/20,9/20/20,9/21/20,9/22/20,9/23/20,9/24/20,9/25/20,9/26/20,9/27/20,9/28/20,9/29/20,9/30/20,10/1/20,10/2/20,10/3/20,10/4/20,10/5/20,10/6/20,10/7/20,10/8/20,10/9/20,10/10/20,10/11/20,10/12/20,10/13/20,stateFips,countyFips
0,84001001,US,USA,840,1001,Autauga,Alabama,US,32.539527,-86.644082,"Autauga, Alabama, US",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,4,6,6,6,6,6,7,8,10,12,12,12,12,12,12,15,17,19,19,19,23,24,26,26,25,26,28,30,32,33,36,36,37,39,40,43,44,42,45,48,53,53,58,61,67,68,74,84,91,93,104,103,110,110,120,127,136,143,149,155,159,168,189,192,205,212,216,221,233,239,239,241,248,259,265,272,282,295,312,323,331,357,364,367,369,394,405,425,428,436,447,463,473,482,492,497,521,530,545,553,560,583,607,610,636,643,653,662,676,698,715,733,743,765,773,810,823,838,845,863,877,891,900,909,932,941,949,963,972,988,1010,1024,1030,1030,1050,1065,1086,1169,1174,1162,1188,1196,1196,1206,1218,1219,1235,1241,1240,1255,1264,1266,1286,1286,1281,1284,1296,1309,1345,1348,1354,1345,1349,1355,1371,1377,1383,1385,1398,1413,1420,1432,1442,1447,1463,1619,1624,1664,1673,1690,1691,1714,1715,1738,1757,1764,1773,1785,1787,1791,1798,1805,1818,1828,1831,1839,1852,1863,1882,1898,1905,1911,1924,01,001
1,84001003,US,USA,840,1003,Baldwin,Alabama,US,30.727750,-87.722071,"Baldwin, Alabama, US",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,2,2,2,3,4,4,5,5,10,15,18,19,20,24,28,29,29,38,42,44,56,59,66,71,72,87,91,101,103,109,112,117,123,132,143,147,147,161,168,171,174,174,175,181,187,188,189,196,205,208,216,222,224,227,231,243,244,254,254,260,262,270,269,271,273,274,274,277,282,281,282,283,290,292,292,292,293,296,304,313,320,325,331,343,353,361,364,368,374,377,386,398,405,415,422,435,449,462,500,539,559,626,663,686,735,828,846,864,894,980,1039,1114,1170,1207,1277,1341,1396,1499,1579,1669,1796,1914,1990,2077,2164,2423,2473,2620,2665,2727,2793,2865,2984,3056,3095,3175,3210,3263,3322,3380,3443,3502,3544,3651,3670,3710,3

#### Reformat dataframe by day

In [12]:
days = county_confirmed.columns.tolist()[11:-2]

In [13]:
timeseries_confirmed = pd.concat((split_by_day(county_confirmed, day, 'cases') for day in days))
timeseries_confirmed['date'] = timeseries_confirmed['date'].apply(dateutil.parser.parse)
timeseries_confirmed['cases'] = timeseries_confirmed['cases'].astype('int')

In [14]:
timeseries_confirmed.shape

(884184, 8)

In [15]:
timeseries_confirmed.head()

,stateFips,countyFips,admin1,admin2,lat,long,cases,date
0,01,001,Alabama,Autauga,32.539527,-86.644082,0,2020-01-22
1,01,003,Alabama,Baldwin,30.727750,-87.722071,0,2020-01-22
2,01,005,Alabama,Barbour,31.868263,-85.387129,0,2020-01-22
3,01,007,Alabama,Bibb,32.996421,-87.125115,0,2020-01-22
4,01,009,Alabama,Blount,33.982109,-86.567906,0,2020-01-22


### Process US cummulative deaths

In [16]:
deaths = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv",  dtype='str')

In [17]:
deaths = deaths.fillna('')

In [18]:
deaths.rename(columns={'Province_State': 'admin1', 'Admin2': 'admin2', 
                       'Lat': 'lat', 'Long_': 'long'}, inplace=True)

In [19]:
deaths['FIPS'] = deaths['FIPS'].str.replace('\.0', '')

Keep only cases with fips codes

In [20]:
county_deaths = deaths.query("admin2 != ''").query("FIPS != ''").copy()

Pad state fips code: 2 character with '0'-padding, e.g. 06

In [21]:
county_deaths['stateFips'] = county_deaths['FIPS'].str[:-3]
county_deaths['stateFips'] = county_deaths['stateFips'].apply(lambda s: '0' + s if len(s) == 1 else s)
county_deaths['countyFips'] = county_deaths['FIPS'].str[-3:]
county_deaths['lat'] = county_deaths['lat'].astype('float')
county_deaths['long'] = county_deaths['long'].astype('float')

In [22]:
county_deaths.tail()

,UID,iso2,iso3,code3,FIPS,admin2,admin1,Country_Region,lat,long,Combined_Key,Population,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20,5/20/20,5/21/20,5/22/20,5/23/20,5/24/20,5/25/20,5/26/20,5/27/20,5/28/20,5/29/20,5/30/20,5/31/20,6/1/20,6/2/20,6/3/20,6/4/20,6/5/20,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20,6/12/20,6/13/20,6/14/20,6/15/20,6/16/20,6/17/20,6/18/20,6/19/20,6/20/20,6/21/20,6/22/20,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20,7/3/20,7/4/20,7/5/20,7/6/20,7/7/20,7/8/20,7/9/20,7/10/20,7/11/20,7/12/20,7/13/20,7/14/20,7/15/20,7/16/20,7/17/20,7/18/20,7/19/20,7/20/20,7/21/20,7/22/20,7/23/20,7/24/20,7/25/20,7/26/20,7/27/20,7/28/20,7/29/20,7/30/20,7/31/20,8/1/20,8/2/20,8/3/20,8/4/20,8/5/20,8/6/20,8/7/20,8/8/20,8/9/20,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20,8/19/20,8/20/20,8/21/20,8/22/20,8/23/20,8/24/20,8/25/20,8/26/20,8/27/20,8/28/20,8/29/20,8/30/20,8/31/20,9/1/20,9/2/20,9/3/20,9/4/20,9/5/20,9/6/20,9/7/20,9/8/20,9/9/20,9/10/20,9/11/20,9/12/20,9/13/20,9/14/20,9/15/20,9/16/20,9/17/20,9/18/20,9/19/20,9/20/20,9/21/20,9/22/20,9/23/20,9/24/20,9/25/20,9/26/20,9/27/20,9/28/20,9/29/20,9/30/20,10/1/20,10/2/20,10/3/20,10/4/20,10/5/20,10/6/20,10/7/20,10/8/20,10/9/20,10/10/20,10/11/20,10/12/20,10/13/20,stateFips,countyFips
3335,84056039,US,USA,840,56039,Teton,Wyoming,US,43.935225,-110.589080,"Teton, Wyoming, US",23464,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,56,039
3336,84056041,US,USA,840,56041,Uinta,Wyoming,US,41.287818,-110.547578,"Uinta, Wyoming, US",20226,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,56,041
3337,84090056,US,USA,840,90056,Unassigned,Wyoming,US,0.000000,0.000000,"Unassigned, Wyoming, US",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,5,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,7,9,9,10,11,11,11,11,11,12,13,14,14,15,15,16,16,16,16,16,16,16,16,16,17,17,17,17,17,17,17,17,17,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,20,20,20,20,20,20,20,21,21,23,23,23,24,23,24,24,24,24,24,24,24,25,25,25,25,25,

#### Reformat dataframe by day

In [23]:
timeseries_deaths = pd.concat((split_by_day(county_deaths, day, 'deaths') for day in days))
timeseries_deaths['date'] = timeseries_deaths['date'].apply(dateutil.parser.parse)
timeseries_deaths['deaths'] = timeseries_deaths['deaths'].astype('int')

In [24]:
timeseries_deaths.shape

(884184, 8)

In [25]:
timeseries_deaths.tail()

,stateFips,countyFips,admin1,admin2,lat,long,deaths,date
3335,56,039,Wyoming,Teton,43.935225,-110.589080,1,2020-10-13
3336,56,041,Wyoming,Uinta,41.287818,-110.547578,2,2020-10-13
3337,90,056,Wyoming,Unassigned,0.000000,0.000000,0,2020-10-13
3338,56,043,Wyoming,Washakie,43.904516,-107.680187,7,2020-10-13
3339,56,045,Wyoming,Weston,43.839612,-104.567488,0,2020-10-13


### Merge global cases and deaths into a timeseries dataframe

In [26]:
# remove lat and long to avoid merging on floating point numbers
subset = timeseries_confirmed.drop(['lat','long'], axis=1)
timeseries = subset.merge(timeseries_deaths, on=['stateFips', 'countyFips', 'admin1', 'admin2', 'date'])

In [27]:
timeseries.shape

(884184, 9)

In [28]:
timeseries.tail()

,stateFips,countyFips,admin1,admin2,cases,date,lat,long,deaths
884179,56,039,Wyoming,Teton,679,2020-10-13,43.935225,-110.589080,1
884180,56,041,Wyoming,Uinta,384,2020-10-13,41.287818,-110.547578,2
884181,90,056,Wyoming,Unassigned,0,2020-10-13,0.000000,0.000000,0
884182,56,043,Wyoming,Washakie,127,2020-10-13,43.904516,-107.680187,7
884183,56,045,Wyoming,Weston,62,2020-10-13,43.839612,-104.567488,0


Example: San Diego Country

In [29]:
timeseries.query("admin2 == 'San Diego'")

,stateFips,countyFips,admin1,admin2,cases,date,lat,long,deaths
231,06,073,California,San Diego,0,2020-01-22,33.034846,-116.736533,0
3555,06,073,California,San Diego,0,2020-01-23,33.034846,-116.736533,0
6879,06,073,California,San Diego,0,2020-01-24,33.034846,-116.736533,0
10203,06,073,California,San Diego,0,2020-01-25,33.034846,-116.736533,0
13527,06,073,California,San Diego,0,2020-01-26,33.034846,-116.736533,0
16851,06,073,California,San Diego,0,2020-01-27,33.034846,-116.736533,0
20175,06,073,California,San Diego,0,2020-01-28,33.034846,-116.736533,0
23499,06,073,California,San Diego,0,2020-01-29,33.034846,-116.736533,0
26823,06,073,California,San Diego,0,2020-01-30,33.034846,-116.736533,0
30147,06,073,California,San Diego,0,2020-01-31,33.034846,-116.736533,0
